In [482]:
from datasets import load_dataset
import json
import polars as pl

In [483]:
import libcst as cst
import re
import random
from openbugger.bugger import Bugger, bugger_example
from time import perf_counter

In [484]:
import libcst as cst
import libcst.matchers as m
from black import format_str, FileMode

class RemoveComments(cst.CSTTransformer):
    def leave_Comment(self, original_node, updated_node):
        return cst.RemovalSentinel.REMOVE

def remove_comments_and_lint(module_str: str) -> str:
    module_cst = cst.parse_module(module_str)
    module_cst_no_comments = module_cst.visit(RemoveComments())
    code_str_no_comments = module_cst_no_comments.code
    # Lint the code using black
    linted_str = format_str(code_str_no_comments, mode=FileMode())
    return linted_str

def compare_modules(module1: str, module2: str) -> bool:
    # Remove comments and lint both modules
    module1_clean = remove_comments_and_lint(module1)
    module2_clean = remove_comments_and_lint(module2)
    # Parse the cleaned code strings back to libcst.Module for deep comparison
    module1_cst_clean = cst.parse_module(module1_clean)
    module2_cst_clean = cst.parse_module(module2_clean)
    return module1_cst_clean.deep_equals(module2_cst_clean)

In [485]:
import libcst as cst
import re
from functools import lru_cache

def is_valid_python(code):
    if len(code) == 0:
        return False
    try:
        cst.parse_module(code)
        return True
    except Exception:
        return False

def is_single_word_line(line):
    words = re.findall(r'\b\w+\b', line)
    return len(words) == 1

def find_enclosed_newlines(input_string):
    enclosed_newlines = []
    enclosing_strings = []
    string_patterns = [r'"[^"\\]*(\\.[^"\\]*)*"', r"'[^'\\]*(\\.[^'\\]*)*'", r'"""(.*?)"""', r"'''(.*?)'''"]
    
    for pattern in string_patterns:
        for match in re.finditer(pattern, input_string, re.DOTALL):
            start, end = match.span()
            inner_newlines = [i for i in range(start, end) if input_string[i] == '\n']
            enclosed_newlines.extend(inner_newlines)
            enclosing_strings.extend([match.group()] * len(inner_newlines))
            
    return enclosed_newlines, enclosing_strings

def split_input_with_respect_to_enclosed_newlines(input_string):
    enclosed_newlines, _ = find_enclosed_newlines(input_string)
    lines = []
    line_start = 0
    for i, char in enumerate(input_string):
        if char == '\n' and i not in enclosed_newlines:
            lines.append(input_string[line_start:i])
            line_start = i+1
    lines.append(input_string[line_start:])  # add the last line
    return lines

def extract_python_blocks(input_string, start=0, intervals=None):
    if intervals is None:
        intervals = {"Python": [], "Non-Python": []}

    lines = split_input_with_respect_to_enclosed_newlines(input_string)
    n = len(lines)

    for i in range(n):
        if not lines[i].strip() or lines[i].lstrip().startswith('#') or is_single_word_line(lines[i]):  
            continue
        for j in range(n-1, i-1, -1):
            if not lines[j].strip() or lines[j].lstrip().startswith('#') :  # ignore empty end lines
                continue
            code = '\n'.join(lines[i:j+1])
            if is_valid_python(code):
                intervals["Python"].append((i+start, j+start))
                if i > 0:
                    intervals["Non-Python"].append((start, i+start-1))
                if j < n-1:
                    remaining = '\n'.join(lines[j+1:])
                    return extract_python_blocks(remaining, j+start+1, intervals)
                return intervals
    if n > 0 and (start, start+n-1) not in intervals["Non-Python"]:
        intervals["Non-Python"].append((start, start+n-1))
    return intervals

import libcst as cst

def cst_module(code):
    try:
        module = cst.parse_module(code)
        return module
    except Exception:
        return None


def extract_strings_from_intervals(input_string, intervals):
    lines = split_input_with_respect_to_enclosed_newlines(input_string)
    non_python_text = ''
    for interval in intervals['Non-Python']:
        start, end = interval
        non_python_text += '\n'.join(lines[start:end + 1]) + '\n'
    return non_python_text.strip()

def extract_python_from_intervals(input_string, intervals):
    lines = split_input_with_respect_to_enclosed_newlines(input_string)
    python_code_list = []
    if len(intervals['Python']) == 0:
        return None
    for interval in intervals['Python']:
        python_code = ''
        start, end = interval
        python_code += '\n'.join(lines[start:end + 1]) + '\n'
        if python_code.strip() == '':
            python_code_list.append(None)
        else:
            python_code_list.append(python_code.strip())
        
    #check there actually is python code
    
    return python_code_list



In [486]:
import libcst as cst
import re
from functools import lru_cache

def is_valid_python(code):
    if len(code) == 0:
        return False
    try:
        cst.parse_module(code)
        return True
    except Exception:
        return False

def is_single_word_line(line):
    words = re.findall(r'\b\w+\b', line)
    return len(words) == 1

def find_enclosed_newlines(input_string):
    enclosed_newlines = []
    enclosing_strings = []
    string_patterns = [r'"[^"\\]*(\\.[^"\\]*)*"', r"'[^'\\]*(\\.[^'\\]*)*'", r'"""(.*?)"""', r"'''(.*?)'''"]
    
    for pattern in string_patterns:
        for match in re.finditer(pattern, input_string, re.DOTALL):
            start, end = match.span()
            inner_newlines = [i for i in range(start, end) if input_string[i] == '\n']
            enclosed_newlines.extend(inner_newlines)
            enclosing_strings.extend([match.group()] * len(inner_newlines))
            
    return enclosed_newlines, enclosing_strings

def split_input_with_respect_to_enclosed_newlines(input_string):
    enclosed_newlines, _ = find_enclosed_newlines(input_string)
    lines = []
    line_start = 0
    for i, char in enumerate(input_string):
        if char == '\n' and i not in enclosed_newlines:
            lines.append(input_string[line_start:i])
            line_start = i+1
    lines.append(input_string[line_start:])  # add the last line
    return lines


    
def cst_module(code):
    try:
        module = cst.parse_module(code)
        return module
    except Exception:
        return None





In [487]:
def clean_code_and_extract(docs):
    code_list = []
    problem_list = []
    data_list = []
    cst_list = []
    bad_code = []
    explanation_list = []
    for code_with_data,code_with_problem,code,explanation in zip(docs["code_with_data"],docs["code_with_problem"],docs["code_only"],docs["explanation_only"]):
        # Remove ```python and ``` at the end 
        good_code = False
        clean_code = re.sub(r'```python', '', code)
        clean_code = re.sub(r'```.*', '', clean_code).strip()

        split_data = code_with_data.split('```python')
        split_problem = code_with_problem.split('```python')
        if len(split_data) > 1 and len(split_problem) > 1:
            data = split_data[0]
            problem = split_problem[0]
            
            try:
                cst_tree = cst.parse_module(clean_code)
                good_code = True
                code_list.append(clean_code)
                cst_list.append(cst_tree)
                data_list.append(data)
                problem_list.append(problem)
                explanation_list.append(explanation)
            except:
                bad_code.append(clean_code)
    return code_list, problem_list, data_list, cst_list, bad_code, explanation_list

In [488]:
docs = load_dataset(f"mhhmm/leetcode-solutions-python", split="train")

Found cached dataset json (C:/Users/Tommaso/.cache/huggingface/datasets/mhhmm___json/mhhmm--leetcode-solutions-python-c6d2758e3a6cc905/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [489]:
code_list, problem_list, data_list, cst_list, bad_code, explanation_list = clean_code_and_extract(docs)

In [490]:
df = pl.DataFrame(data = { "problem": problem_list,"code": code_list, "cst": [str(x) for x in cst_list]})
df.head()

problem,code,cst
str,str,str
"""# Given an arr…","""def twoSum(num…","""Module( bo…"
"""# You are give…","""class ListNode…","""Module( bo…"
"""# Given a stri…","""def length_of_…","""Module( bo…"
"""# Given two so…","""def findMedian…","""Module( bo…"
"""# Given a stri…","""def longest_pa…","""Module( bo…"


In [491]:
def load_bugs(url: str):
    with open(url, 'r') as file:
        all_bugs = json.load(file)
    return all_bugs

In [492]:
all_bugs_url = 'bugger_all_bugs_leetcode.json'
output_dir = "./outs"
bugs = load_bugs(all_bugs_url)

In [493]:
all_bugs_names = set()
for id, bug_dict in sorted(bugs.items(), key=lambda x: int(x[0])):
    for key in bug_dict.keys():
        all_bugs_names.add(key)
bug_names_code =[x+"_code" for x in all_bugs_names]
bug_names_bugs = [x+"_bugs" for x in all_bugs_names]
all_bugs_ids = bug_names_code + bug_names_bugs

In [494]:
print(all_bugs_names)

{'ReturningEarlyTransformer', 'SwapForTransformer', 'VariableNameTypoTransformer', 'ForgettingToUpdateVariableTransformer', 'MutableDefaultArgumentTransformer', 'UseBeforeDefinitionTransformer', 'OffByKIndexTransformer', 'ComparisonSwapTransformer', 'InfiniteWhileTransformer', 'MissingArgumentTransformer', 'IncorrectExceptionHandlerTransformer', 'IncorrectTypeTransformer', 'ComparisonTargetTransformer', 'IncorrectVariableInitializationTransformer', 'NonExistingMethodTransformer'}


In [ ]:
#bugged dict keys 
bugged_dict = {x:[] for x in all_bugs_ids}
for id, bug_dict in sorted(bugs.items(), key=lambda x: int(x[0])):
    for bug in all_bugs_names:
        if bug in bug_dict.keys():
            bugged_dict[bug+"_code"].append(bug_dict[bug]["bugged_code"])
            bugged_dict[bug+"_bugs"].append(bug_dict[bug]["bugs"])
        else:
            bugged_dict[bug+"_code"].append(None)
            bugged_dict[bug+"_bugs"].append(None)

In [ ]:
df_bugs = pl.DataFrame(data=bugged_dict)

In [ ]:
df_bugs.shape

In [ ]:
bugs["0"]["ForgettingToUpdateVariableTransformer"].keys()

In [ ]:
#get a lit of al json in output_dir
bug_names_user =[x+"_user" for x in all_bugs_names]
bug_names_assistant = [x+"_assistant" for x in all_bugs_names]
all_bugs_conversation_ids= bug_names_user + bug_names_assistant
conversation_dict = {x:[] for x in all_bugs_conversation_ids}

In [ ]:


import os
for i in range(2348):
    for bugtype in all_bugs_names:
        file_name = f"{i}_{bugtype}.json"
        file_path = os.path.join(output_dir, file_name)
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                conversation = json.load(file)
            if isinstance(conversation,dict) and "question" in conversation.keys():
                conversation_dict[bugtype+"_user"].append(conversation["question"])
                conversation_dict[bugtype+"_assistant"].append(conversation["answer"])
            else:
                print("bad conversation", file_path)
                conversation_dict[bugtype+"_user"].append(None)
                conversation_dict[bugtype+"_assistant"].append(None)
        else:
            conversation_dict[bugtype+"_user"].append(None)
            conversation_dict[bugtype+"_assistant"].append(None)

In [ ]:
df_conversation = pl.DataFrame(data=conversation_dict)


In [ ]:
df_complete = pl.concat([df,df_bugs,df_conversation], how= "horizontal")

In [479]:
df_complete.head()


shape: (5, 63)
┌─────────┬─────────┬─────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ problem ┆ code    ┆ cst     ┆ ReturningE ┆ … ┆ IncorrectT ┆ Comparison ┆ IncorrectV ┆ NonExistin │
│ ---     ┆ ---     ┆ ---     ┆ arlyTransf ┆   ┆ ypeTransfo ┆ TargetTran ┆ ariableIni ┆ gMethodTra │
│ str     ┆ str     ┆ str     ┆ ormer_code ┆   ┆ rmer_assis ┆ sformer_as ┆ tializatio ┆ nsformer_a │
│         ┆         ┆         ┆ ---        ┆   ┆ ta…        ┆ si…        ┆ nT…        ┆ ss…        │
│         ┆         ┆         ┆ str        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│         ┆         ┆         ┆            ┆   ┆ str        ┆ str        ┆ str        ┆ str        │
╞═════════╪═════════╪═════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ # Given ┆ def two ┆ Module( ┆ def twoSum ┆ … ┆ null       ┆ Sure, I    ┆ null       ┆ null       │
│ an      ┆ Sum(num ┆ body=[  ┆ (nums,     ┆   ┆            ┆ can help   ┆            ┆            │
│ array   ┆ s, targ ┆ Funct…  ┆ target):   ┆   ┆            ┆ you with   ┆            ┆            │
│ of inte ┆ et):    ┆         ┆     ma…    ┆   ┆            ┆ this. …    ┆            ┆            │
│ gers    ┆     ma… ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
│ `nu…    ┆         ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
│ # You   ┆ class   ┆ Module( ┆ class      ┆ … ┆ Sure, I'll ┆ null       ┆ 1. The     ┆ null       │
│ are     ┆ ListNod ┆ body=[  ┆ ListNode:  ┆   ┆ be happy   ┆            ┆ ListNode   ┆            │
│ given   ┆ e:      ┆ Class…  ┆ def        ┆   ┆ to help!   ┆            ┆ class is a ┆            │
│ two **n ┆ def __i ┆         ┆ __init__…  ┆   ┆            ┆            ┆ simpl…     ┆            │
│ on-empt ┆ nit__…  ┆         ┆            ┆   ┆ 1.…        ┆            ┆            ┆            │
│ y*…     ┆         ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
│ # Given ┆ def len ┆ Module( ┆ def length ┆ … ┆ The        ┆ null       ┆ Sure, I    ┆ This code  │
│ a       ┆ gth_of_ ┆ body=[  ┆ _of_longes ┆   ┆ purpose of ┆            ┆ can help   ┆ is         │
│ string  ┆ longest ┆ Funct…  ┆ t_substrin ┆   ┆ the code   ┆            ┆ you        ┆ attempting │
│ `s`,    ┆ _substr ┆         ┆ g(…        ┆   ┆ is to fi…  ┆            ┆ understand ┆ to solve…  │
│ find    ┆ ing(…   ┆         ┆            ┆   ┆            ┆            ┆ …          ┆            │
│ the l…  ┆         ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
│ # Given ┆ def fin ┆ Module( ┆ def findMe ┆ … ┆ This code  ┆ null       ┆ null       ┆ null       │
│ two     ┆ dMedian ┆ body=[  ┆ dianSorted ┆   ┆ is trying  ┆            ┆            ┆            │
│ sorted  ┆ SortedA ┆ Funct…  ┆ Arrays(num ┆   ┆ to find    ┆            ┆            ┆            │
│ arrays  ┆ rrays(n ┆         ┆ s1…        ┆   ┆ the …      ┆            ┆            ┆            │
│ `nums1… ┆ ums1…   ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
│ # Given ┆ def lon ┆ Module( ┆ def longes ┆ … ┆ This code  ┆ null       ┆ null       ┆ null       │
│ a       ┆ gest_pa ┆ body=[  ┆ t_palindro ┆   ┆ is impleme ┆            ┆            ┆            │
│ string  ┆ lindrom ┆ Funct…  ┆ mic_substr ┆   ┆ nting an   ┆            ┆            ┆            │
│ `s`,    ┆ ic_subs ┆         ┆ in…        ┆   ┆ alg…       ┆            ┆            ┆            │
│ return  ┆ trin…   ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
│ _th…    ┆         ┆         ┆            ┆   ┆            ┆            ┆            ┆            │
└─────────┴─────────┴─────────┴────────────┴───┴────────────┴────────────┴────────────┴────────────┘

In [495]:
#save to arro
df_complete.write_parquet("df_complete_leetcode_dirty.parquet")